In [2]:
import streamlit as st
import json
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [3]:
def load_data():
    try:
        with open('sales_chatbot_data.json', 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        st.error("No data file found. Please run the input app first to create the data.")
        return None
    

In [4]:
data=load_data()
data

{'product_name': 'EcoClean All-Purpose Cleaner',
 'product_description': 'EcoClean is a powerful, eco-friendly all-purpose cleaning solution. Made from 100% natural ingredients, it effectively cleans surfaces while being safe for your family and the environment. EcoClean works on multiple surfaces including countertops, floors, bathrooms, and kitchen appliances.',
 'pricing_details': '500ml bottle: $9.99\n1L bottle: $16.99\n2L refill pack: $29.99\nSubscription (1L bottle every month): $14.99/month\nBulk order (5 x 1L bottles): $74.99 (Save $10!)',
 'faqs': [{'question': 'Is EcoClean safe to use around children and pets?',
   'answer': 'Yes, EcoClean is made from 100% natural ingredients and is non-toxic, making it safe to use around children and pets. However, as with any cleaning product, we recommend keeping it out of reach of children and avoiding direct contact with eyes or ingestion.'}],
 'sales_script': 'Keep it Short massage , understand the problem and then promote this product

In [5]:
llm = ChatGroq(model="llama-3.2-11b-vision-preview", api_key="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")

In [6]:
prompt_template = PromptTemplate(
    input_variables=["product_info", "chat_history", "human_input", "links_info", "tone", "language", "objections", "faqs", "sales_script"],
    template="""
    You are an AI sales representative for the following product:
    {product_info}
    
    Your tone should be {tone} and you should communicate in {language}.
    
    Here are some common objections and how to handle them:
    {objections}
    
    Here are some frequently asked questions and their answers:
    {faqs}
    
    Use the following sales script as a guideline:
    {sales_script}

    Important: Only provide the following links if the user specifically asks for them or indicates they're ready to buy:
    {links_info}
    
        
    Human: {human_input}
    AI: """
)

In [7]:
from langchain.chains.llm import LLMChain
llm_chain=LLMChain(llm=llm,prompt=prompt_template)


C:\Users\Akshay Kumar BM\AppData\Local\Temp\ipykernel_7924\4249127722.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain=LLMChain(llm=llm,prompt=prompt_template)


In [10]:
product_info = f"""
        Product Name: {data.get('product_name', 'N/A')}
        Description: {data.get('product_description', 'N/A')}
        Pricing: {data.get('pricing_details', 'N/A')}
        """
objections = "\n".join([f"Objection: {obj.get('objection', 'N/A')}\nResponse: {obj.get('response', 'N/A')}" for obj in data.get('objections', [])])

faqs = "\n".join([f"Q: {faq.get('question', 'N/A')}\nA: {faq.get('answer', 'N/A')}" for faq in data.get('faqs', [])])

       
links_info = f"""
        Purchase Link: {data.get('purchase_link', 'Not available')}
        Product Demo Link: {data.get('product_demo_link', 'Not available')}
        Appointment Booking Link: {data.get('appointment_booking_link', 'Not available')}"""
tone=data['tone']
language=data['language']
sales_script=data['sales_script']


In [24]:
human="i need very cheap"

In [25]:
llm_chain.invoke(input={'language':language, 'sales_script':sales_script, 'tone':tone, 'faqs':faqs, 'human_input':human, 'links_info':links_info, 'chat_history':"", 'product_info':product_info, 'objections':objections})

{'language': 'English ',
 'sales_script': 'Keep it Short massage , understand the problem and then promote this product ',
 'tone': 'Formal',
 'faqs': 'Q: Is EcoClean safe to use around children and pets?\nA: Yes, EcoClean is made from 100% natural ingredients and is non-toxic, making it safe to use around children and pets. However, as with any cleaning product, we recommend keeping it out of reach of children and avoiding direct contact with eyes or ingestion.',
 'human_input': 'i need very cheap',
 'links_info': '\n        Purchase Link: https://www.ecoclean-example.com/shop\n        Product Demo Link: https://www.ecoclean-example.com/demo-video\n        Appointment Booking Link: https://www.ecoclean-example.com/book-consultation',
 'chat_history': '',
 'product_info': '\n        Product Name: EcoClean All-Purpose Cleaner\n        Description: EcoClean is a powerful, eco-friendly all-purpose cleaning solution. Made from 100% natural ingredients, it effectively cleans surfaces while 